In [1]:
from FormulaTools import *
from Progression import *
from Constants import *
from SpecificationMDPTools import *
import numpy as np

PathToDataFile = ''
SampleSignal = ImportSampleData(PathToDataFile)


Formulas, Prob = ImportCandidateFormulas()
idx = np.argsort(Prob)[::-1]
Formulas = np.array(Formulas)
Prob = np.array(Prob)
Formulas = Formulas[idx]
Prob = Prob[idx]
formula = Formulas[0]
print(formula)

['and', ['G', ['and', ['not', ['T0']], ['not', ['T1']], ['not', ['T2']], ['not', ['T3']], ['not', ['T4']]]], ['imp', ['P0'], ['F', ['W0']]], ['imp', ['P1'], ['F', ['W1']]], ['imp', ['P2'], ['F', ['W2']]], ['imp', ['P3'], ['F', ['W3']]], ['imp', ['P0'], ['U', ['not', ['W1']], ['W0']]], ['imp', ['P0'], ['U', ['not', ['W2']], ['W0']]], ['imp', ['P1'], ['U', ['not', ['W2']], ['W1']]]]


Now first lets verify that the syntax of the sampled formulas is correct

In [2]:
print(VerifyFormulaSyntax(formula))

(True, 'AndOr')


Now lets check that the vocabulary of the formula is a subset of the signal vocabulary

In [3]:
print('Formula Vocabulary', GetVocabulary(formula, set()))
print('Signal predicates', set(SampleSignal.keys()) - set(['length']))
print(VerifyVocabulary(formula, SampleSignal))

Formula Vocabulary {'P3', 'W3', 'T2', 'P0', 'T0', 'W2', 'P1', 'T3', 'W1', 'T4', 'P2', 'W0', 'T1'}
Signal predicates {'P3', 'W3', 'T2', 'P0', 'T0', 'W2', 'P1', 'W1', 'T3', 'T4', 'P2', 'W0', 'T1'}
True


Lets generate a slice of the labeling propositions at the first time step

In [4]:
SignalSlice = GetTraceSlice(SampleSignal,0)
print(SignalSlice)

{'P3': True, 'W3': False, 'T2': False, 'P0': True, 'T0': False, 'W2': False, 'P1': True, 'W1': False, 'T3': False, 'T4': False, 'P2': True, 'W0': False, 'T1': False}



The formulas have conditional predicates that make their effect felt from the first time step and are not relevant in the dynamic part of the task, so lets progress the formula by a single time step before beginning the task

In [5]:
progressed_formula = ProgressSingleTimeStep(formula, SignalSlice)
print(progressed_formula)


['and', ['G', ['and', ['not', ['T0']], ['not', ['T1']], ['not', ['T2']], ['not', ['T3']], ['not', ['T4']]]], ['F', ['W0']], ['F', ['W1']], ['F', ['W2']], ['F', ['W3']], ['U', ['not', ['W1']], ['W0']], ['U', ['not', ['W2']], ['W0']], ['U', ['not', ['W2']], ['W1']]]


Note that the progressed formula has a more limited vocabulary compared to the original formula and this is better for progression.

In [6]:
GetVocabulary(progressed_formula, set())

{'T0', 'T1', 'T2', 'T3', 'T4', 'W0', 'W1', 'W2', 'W3'}

We can now use `FindAllProgressions_single_formula` to evaluate all possible progressions of the given formula and a description of the graph of formulas given the truth assignments in the vocabulary

In [7]:
progressions, edges = FindAllProgressions_single_formula(progressed_formula)
states = dict([(v,k) for (k,v) in progressions.items()])

Note that as the LTL formulas were stored as nested lists, we have to convert them to json strings to use a dictionary to hash them. `progressions` is a dictionary where the keys represent all possible progressions of the original formula. The value stored at the key represents a unique integer index that can be used as a one-hot encoding 

In [8]:
key = list(progressions.keys())[0]
print(key)

["and", ["G", ["and", ["not", ["T0"]], ["not", ["T1"]], ["not", ["T2"]], ["not", ["T3"]], ["not", ["T4"]]]], ["F", ["W0"]], ["F", ["W1"]], ["F", ["W2"]], ["F", ["W3"]], ["U", ["not", ["W1"]], ["W0"]], ["U", ["not", ["W2"]], ["W0"]], ["U", ["not", ["W2"]], ["W1"]]]


`states` is the inverse map of the states to the LTL formula representing the state

In [9]:
states[0]==key

True

`edges` is a tuple structure that represents the graph of transitions between the LTL formulas and the truth value assignments that cause that transitions

In [10]:
edge = edges[5]
print(f'The source state/formula is:\n {edge[0]}\n\n')
print(f'With truth assignment: \n{edge[2]}\n\n')
print(f'The resulting formula is: \n {edge[1]}')

The source state/formula is:
 ['and', ['G', ['and', ['not', ['T0']], ['not', ['T1']], ['not', ['T2']], ['not', ['T3']], ['not', ['T4']]]], ['F', ['W0']], ['F', ['W1']], ['F', ['W2']], ['F', ['W3']], ['U', ['not', ['W1']], ['W0']], ['U', ['not', ['W2']], ['W0']], ['U', ['not', ['W2']], ['W1']]]


With truth assignment: 
[{'T0': False, 'T1': False, 'T2': False, 'T3': False, 'T4': False, 'W0': True, 'W1': False, 'W2': False, 'W3': True}]


The resulting formula is: 
 ['and', ['G', ['and', ['not', ['T0']], ['not', ['T1']], ['not', ['T2']], ['not', ['T3']], ['not', ['T4']]]], ['F', ['W1']], ['F', ['W2']], ['U', ['not', ['W2']], ['W1']]]


Note that both 'W1' and 'W2' were completed simultaneously


Once the progression states are known, we can use `FindTerminalStates_single_formula` to identify the progressions that can serve as the end of a demonstration. The terminal states are when the formula is either progressed to `[True]` or to a cosafe formula

In [11]:
terminal_states = FindTerminalStates_single_formula(progressions)
print(terminal_states)

['[false]', '["G", ["and", ["not", ["T0"]], ["not", ["T1"]], ["not", ["T2"]], ["not", ["T3"]], ["not", ["T4"]]]]']


Now lets consider an example from the dinner table domain

In [13]:
import json
Data = json.load(open('DinnerTable_OutputDist_Sampler4_Custom_30.json','r'))
formula = Data['support'][1]
Signal = ImportBSIData('DinnerTableData.json')
TraceSlice = GetTraceSlice(Signal, 0)
progressed_formula = ProgressSingleTimeStep(formula, TraceSlice)

Note that the original formula contains 20 propositions and 3 orderings, that represents a pretty large set of formulas that can be progressed to including all possible evaluations of the conditionals.

In [14]:
Progressions, Edges = FindAllProgressions_single_formula(formula)
for val in Progressions.keys():
    print(json.loads(val), ': ', Progressions[val], '\n')

['and', ['G', ['not', ['T0']]], ['imp', ['P0'], ['F', ['W0']]], ['imp', ['P1'], ['F', ['W1']]], ['imp', ['P2'], ['F', ['W2']]], ['imp', ['P3'], ['F', ['W3']]], ['imp', ['P5'], ['F', ['W5']]], ['imp', ['P6'], ['F', ['W6']]], ['imp', ['P1'], ['U', ['not', ['W2']], ['W1']]], ['imp', ['P3'], ['U', ['not', ['W6']], ['W3']]], ['imp', ['P7'], ['U', ['not', ['W5']], ['W7']]]] :  0 

[False] :  1 

['G', ['not', ['T0']]] :  2 

['and', ['G', ['not', ['T0']]], ['F', ['W6']]] :  3 

['and', ['G', ['not', ['T0']]], ['F', ['W5']]] :  4 

['and', ['G', ['not', ['T0']]], ['F', ['W5']], ['U', ['not', ['W5']], ['W7']]] :  5 

['and', ['G', ['not', ['T0']]], ['F', ['W5']], ['F', ['W6']]] :  6 

['and', ['G', ['not', ['T0']]], ['F', ['W5']], ['F', ['W6']], ['U', ['not', ['W5']], ['W7']]] :  7 

['and', ['G', ['not', ['T0']]], ['F', ['W3']], ['F', ['W6']], ['U', ['not', ['W6']], ['W3']]] :  8 

['and', ['G', ['not', ['T0']]], ['F', ['W3']], ['F', ['W5']], ['F', ['W6']], ['U', ['not', ['W6']], ['W3']]] :  

Now lets compare this to what happens when we progress it by a single time step for a given scenario

In [16]:
Progressions_one_step, Edges_one_step = FindAllProgressions_single_formula(progressed_formula)
for val in Progressions_one_step.keys():
    print(json.loads(val), ': ', Progressions_one_step[val], '\n')

['and', ['G', ['not', ['T0']]], ['F', ['W0']], ['F', ['W2']], ['F', ['W5']]] :  0 

[False] :  1 

['G', ['not', ['T0']]] :  2 

['and', ['G', ['not', ['T0']]], ['F', ['W5']]] :  3 

['and', ['G', ['not', ['T0']]], ['F', ['W2']]] :  4 

['and', ['G', ['not', ['T0']]], ['F', ['W2']], ['F', ['W5']]] :  5 

['and', ['G', ['not', ['T0']]], ['F', ['W0']]] :  6 

['and', ['G', ['not', ['T0']]], ['F', ['W0']], ['F', ['W5']]] :  7 

['and', ['G', ['not', ['T0']]], ['F', ['W0']], ['F', ['W2']]] :  8 



This should be a subset of all possible progressions of the original formula. Lets check that

In [17]:
Set1 = set(Progressions.keys())
Set2 = set(Progressions_one_step.keys())
print(Set2.issubset(Set1))

True


The output of Bayesian specification inference is not a single formula but a cateforical distribution over a collection of formulas. Thus the progression state is not a formula, but a collection of formulas consisting of different progression stage of each of the component formulas. Lets see how to get these progression states for a list of 150 top formulas in the synthetic domain dataset. Lets reload the dataset first

In [19]:
PathToDataFile = ''
SampleSignal = ImportSampleData(PathToDataFile)
TraceSlice = GetTraceSlice(SampleSignal,0)
Formulas, Prob = ImportCandidateFormulas()
idx = np.argsort(Prob)[::-1]
Formulas = np.array(Formulas)[idx]
Prob = np.array(Prob)[idx]
ProgressedFormulas = np.array([ProgressSingleTimeStep(formula, TraceSlice) for formula in Formulas])


Now lets find all possible progression states for MAP formula in the distribution. This should be the same as the case above

In [20]:
progression_states_speclist, edges_speclist = FindAllProgressions([ProgressedFormulas[0]])
states_speclist = set([key[0] for key in progression_states_speclist.keys()])

progression_states_singleformula, edges_singleformula = FindAllProgressions_single_formula(ProgressedFormulas[0])
states_singleformula = set(progression_states_singleformula.keys())

print('The number of progression states: ', len(progression_states_speclist))
states_speclist == states_singleformula

The number of progression states:  9


True

This has the same 9 progression states as the single formula. Now lets see what happens when we start including more formulas into the specification set

In [22]:
progression_states, edges = FindAllProgressions(ProgressedFormulas[0:5])
print('The number of progression states: ', len(progression_states))

The number of progression states:  17


In [24]:
for i in [5, 10, 50, 100, 150,178]:
    progression_states, edges = FindAllProgressions(ProgressedFormulas[0:i])
    print(f'The cumulative probability with {i} formulas is: ', np.sum(Prob[0:i]))
    print(f'The number of progression states with {i} formulas is: ', len(progression_states),'\n')

The cumulative probability with 5 formulas is:  0.454066666667
The number of progression states with 5 formulas is:  17 

The cumulative probability with 10 formulas is:  0.541866666667
The number of progression states with 10 formulas is:  25 

The cumulative probability with 50 formulas is:  0.838866666667
The number of progression states with 50 formulas is:  79 

The cumulative probability with 100 formulas is:  0.949133333333
The number of progression states with 100 formulas is:  135 

The cumulative probability with 150 formulas is:  0.993266666667
The number of progression states with 150 formulas is:  183 

The cumulative probability with 178 formulas is:  1.0
The number of progression states with 178 formulas is:  237 



For a few different formulas in support lets compare this by constructing the progression states naively

In [26]:
Vals = [1,2,3,4,5,6,7]


for nFormulas in Vals:
    individual_progression_states = {}
    # Do the naive state enumeration
    for formula in ProgressedFormulas[0:nFormulas]:
        individual_progression_states[json.dumps(formula)], _  = FindAllProgressions_single_formula(formula)

    from itertools import product

    progression_states_naive = list(product(*tuple([list(individual_progression_states[key].keys()) for key in individual_progression_states.keys()])))
    print(f'The number of naive progression states with {nFormulas} formulas is: ', len(progression_states_naive))
    
    #Do the smart enumeration
    progression_states, _ = FindAllProgressions(ProgressedFormulas[0:nFormulas])
    print(f'The number of possible progression states with {nFormulas} formulas is: ', len(progression_states), '\n')
    #check if the list of progression states we compute is a subset of the ones computed naively (should be)

    set(progression_states.keys()).issubset(set(progression_states_naive))

The number of naive progression states with 1 formulas is:  9
The number of possible progression states with 1 formulas is:  9 

The number of naive progression states with 2 formulas is:  63
The number of possible progression states with 2 formulas is:  11 

The number of naive progression states with 3 formulas is:  441
The number of possible progression states with 3 formulas is:  11 

The number of naive progression states with 4 formulas is:  3969
The number of possible progression states with 4 formulas is:  11 

The number of naive progression states with 5 formulas is:  27783
The number of possible progression states with 5 formulas is:  17 

The number of naive progression states with 6 formulas is:  194481
The number of possible progression states with 6 formulas is:  17 

The number of naive progression states with 7 formulas is:  1750329
The number of possible progression states with 7 formulas is:  25 



Clearly naively listing all possible progressions is a terrible idea. Our approach for a well constructed specification sets will settle on a reasonable number of progression states quite well. Lets also verify that this approach also works with the dinner table domain example. The dinner table formulas has fewer ordering constraint thus it has more possible progression states per formula. We start by loading the dataset

In [28]:
Data = json.load(open('DinnerTable_OutputDist_Sampler4_Custom_30.json','r'))
Formulas, Probs = Data['support'],Data['probs']
#Sort the formulas as per the probabilities
idx = np.argsort(Probs)[::-1]
Formulas = np.array(Formulas)[idx]
Probs = np.array(Probs)[idx]

#Proposition data
Signal = ImportBSIData('DinnerTableData.json')
TraceSlice = GetTraceSlice(Signal, 0)
Conditions = [key for key in TraceSlice.keys() if 'P' in key]
for key in Conditions: TraceSlice[key] = True


#Progress every formula by a single time step
ProgressedFormulas = np.array([ProgressSingleTimeStep(formula, TraceSlice) for formula in Formulas])
print('The number of formulas in the posterior is: ', len(ProgressedFormulas))





The number of formulas in the posterior is:  25


Now lets check if for a single formulas this gives the correct answer

In [29]:
#Check with single progression
progression_states_single_formula, edges_single_formula = FindAllProgressions_single_formula(ProgressedFormulas[0])
Set1 = set(progression_states_single_formula)

#Use set progression and verify that for a single formula the sets are equal
progression_states_spec_list, edges_spec_list = FindAllProgressions([ProgressedFormulas[0]])
Set2 = set([key[0] for key in progression_states_spec_list.keys()])

Set1 == Set2

True

Now lets check the progression states sizes for different formula sets in the specification list

In [30]:
nFormulas = [1,5,10,15,20,25]

for nForm in nFormulas:
    print(f'The cumulative probability with {nForm} formulas is: ', np.sum(Probs[0:nForm]))
    progression_states, edges = FindAllProgressions(ProgressedFormulas[0:nForm])
    print(f'The number of unique progression states with {nForm} formulas is: ', len(progression_states),'\n')




The cumulative probability with 1 formulas is:  0.276133333333
The number of unique progression states with 1 formulas is:  193 

The cumulative probability with 5 formulas is:  0.942533333333
The number of unique progression states with 5 formulas is:  521 

The cumulative probability with 10 formulas is:  0.971533333333
The number of unique progression states with 10 formulas is:  1155 

The cumulative probability with 15 formulas is:  0.9894
The number of unique progression states with 15 formulas is:  1437 

The cumulative probability with 20 formulas is:  0.997933333333
The number of unique progression states with 20 formulas is:  2023 

The cumulative probability with 25 formulas is:  1.0
The number of unique progression states with 25 formulas is:  3024 



Note that we can compare the sizes of the naive and the actual number of progression states for this too, however even with just 2 candidate formulas, we run out of memory.

Lets determine the terminal states in both the synthetic domain and the dinner table domain with all the formulas in the posterior considered. Lets reload the dataset with the scoped names

In [31]:
PathToDataFile = ''
SampleSignal = ImportSampleData(PathToDataFile)
TraceSlice = GetTraceSlice(SampleSignal,0)
Formulas, Prob = ImportCandidateFormulas()
idx = np.argsort(Prob)[::-1]
Formulas_synth = np.array(Formulas)[idx]
Probs_synth = np.array(Prob)[idx]
ProgressedFormulas_synth = np.array([ProgressSingleTimeStep(formula, TraceSlice) for formula in Formulas_synth])

Data = json.load(open('DinnerTable_OutputDist_Sampler4_Custom_30.json','r'))
Formulas, Probs = Data['support'],Data['probs']
#Sort the formulas as per the probabilities
idx = np.argsort(Probs)[::-1]
Formulas_dinner = np.array(Formulas)[idx]
Probs_dinner = np.array(Probs)[idx]

#Proposition data
Signal = ImportBSIData('DinnerTableData.json')
TraceSlice = GetTraceSlice(Signal, 0)
Conditions = [key for key in TraceSlice.keys() if 'P' in key]
for key in Conditions: TraceSlice[key] = True


#Progress every formula by a single time step
ProgressedFormulas_dinner = np.array([ProgressSingleTimeStep(formula, TraceSlice) for formula in Formulas_dinner])
#print('The number of formulas in the posterior is: ', len(ProgressedFormulas_dinner))

ProgressedFormulas = {'synth': ProgressedFormulas_synth, 'dinner': ProgressedFormulas_dinner}
Probs = {'synth': Probs_synth, 'dinner': Probs_dinner}
Formulas = {'synth': Formulas_synth, 'dinner': Formulas_dinner}

progression_states = {}
edges = {}
terminal_states = {}
Domain = {'synth': 'Synthetic Domain', 'dinner': 'Dinner Table Domain'}

keys = ['synth', 'dinner']
for key in keys:
    domain = Domain[key]
    print(f'Finding progression states for {domain}')
    progression_states[key], edges[key] = FindAllProgressions(ProgressedFormulas[key])
    terminal_states[key] = FindTerminalStates(progression_states[key])
    print(f'Number of unique progressions in {domain}: ', len(progression_states[key]))
    print(f'Number of edges in {domain}: ', len(edges[key]))
    print(f'Number of terminal states in {domain}: ', len(terminal_states[key]),'\n')

Finding progression states for Synthetic Domain
Number of unique progressions in Synthetic Domain:  237
Number of edges in Synthetic Domain:  3190
Number of terminal states in Synthetic Domain:  62 

Finding progression states for Dinner Table Domain
Number of unique progressions in Dinner Table Domain:  3024
Number of edges in Dinner Table Domain:  26562
Number of terminal states in Dinner Table Domain:  326 

